In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')
        self.test_x_f1 = np.load('../testImage_canny.npy')
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.x_f1: batch_x_f1_test,
                                                                   self.x_f2: batch_x_f2_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [3]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# Avg per epoch ptime: 14.41, total 50 epochs ptime: 739.12
#  [*] Training finished!
#  [*] Best Epoch:  17 , Accuracy:  0.7200000286102295
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-17
#  [*] Finished testing Best Epoch: 17 , accuracy:  0.7200000286102295 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32

Epoch: [ 3] [  15/  40] time: 36.7795, loss: 0.01931938
Epoch: [ 3] [  16/  40] time: 37.1171, loss: 0.07564691
Epoch: [ 3] [  17/  40] time: 37.4553, loss: 0.06133568
Epoch: [ 3] [  18/  40] time: 37.7950, loss: 0.06982832
Epoch: [ 3] [  19/  40] time: 38.1317, loss: 0.15041555
Epoch: [ 3] [  20/  40] time: 38.4696, loss: 0.10949417
Epoch: [ 3] [  21/  40] time: 38.8076, loss: 0.11697094
Epoch: [ 3] [  22/  40] time: 39.1458, loss: 0.09776273
Epoch: [ 3] [  23/  40] time: 39.4858, loss: 0.06171544
Epoch: [ 3] [  24/  40] time: 39.8258, loss: 0.03051960
Epoch: [ 3] [  25/  40] time: 40.1633, loss: 0.05230125
Epoch: [ 3] [  26/  40] time: 40.5003, loss: 0.06947006
Epoch: [ 3] [  27/  40] time: 40.8382, loss: 0.11566653
Epoch: [ 3] [  28/  40] time: 41.1754, loss: 0.05329986
Epoch: [ 3] [  29/  40] time: 41.5132, loss: 0.11711921
Epoch: [ 3] [  30/  40] time: 41.8517, loss: 0.09056684
Epoch: [ 3] [  31/  40] time: 42.1895, loss: 0.03608584
Epoch: [ 3] [  32/  40] time: 42.5293, loss: 0.1

Epoch: [ 6] [  38/  40] time: 88.7075, loss: 0.00769896
Epoch: [ 6] [  39/  40] time: 89.0461, loss: 0.01794152
[6/50] - ptime: 14.3331 loss: 0.05133857 acc: 0.27333 lr: 0.00100000
Epoch: [ 7] [   0/  40] time: 90.5424, loss: 0.03432508
Epoch: [ 7] [   1/  40] time: 90.8820, loss: 0.00881518
Epoch: [ 7] [   2/  40] time: 91.2220, loss: 0.01167565
Epoch: [ 7] [   3/  40] time: 91.5621, loss: 0.04879512
Epoch: [ 7] [   4/  40] time: 91.9019, loss: 0.00742390
Epoch: [ 7] [   5/  40] time: 92.2393, loss: 0.03619664
Epoch: [ 7] [   6/  40] time: 92.5776, loss: 0.01145342
Epoch: [ 7] [   7/  40] time: 92.9169, loss: 0.00967853
Epoch: [ 7] [   8/  40] time: 93.2551, loss: 0.02505825
Epoch: [ 7] [   9/  40] time: 93.5939, loss: 0.00836917
Epoch: [ 7] [  10/  40] time: 93.9332, loss: 0.04146182
Epoch: [ 7] [  11/  40] time: 94.2722, loss: 0.01385154
Epoch: [ 7] [  12/  40] time: 94.6105, loss: 0.01321753
Epoch: [ 7] [  13/  40] time: 94.9485, loss: 0.03542954
Epoch: [ 7] [  14/  40] time: 95.28

Epoch: [10] [  18/  40] time: 140.8528, loss: 0.01368716
Epoch: [10] [  19/  40] time: 141.1916, loss: 0.00148998
Epoch: [10] [  20/  40] time: 141.5317, loss: 0.00069934
Epoch: [10] [  21/  40] time: 141.8718, loss: 0.00885996
Epoch: [10] [  22/  40] time: 142.2111, loss: 0.00163721
Epoch: [10] [  23/  40] time: 142.5508, loss: 0.01473086
Epoch: [10] [  24/  40] time: 142.8897, loss: 0.00532676
Epoch: [10] [  25/  40] time: 143.2289, loss: 0.01338995
Epoch: [10] [  26/  40] time: 143.5690, loss: 0.01941705
Epoch: [10] [  27/  40] time: 143.9075, loss: 0.04272924
Epoch: [10] [  28/  40] time: 144.2478, loss: 0.03325539
Epoch: [10] [  29/  40] time: 144.5868, loss: 0.02379362
Epoch: [10] [  30/  40] time: 144.9266, loss: 0.03208477
Epoch: [10] [  31/  40] time: 145.2669, loss: 0.00880742
Epoch: [10] [  32/  40] time: 145.6072, loss: 0.01945541
Epoch: [10] [  33/  40] time: 145.9483, loss: 0.00685188
Epoch: [10] [  34/  40] time: 146.2885, loss: 0.01356321
Epoch: [10] [  35/  40] time: 1

Epoch: [13] [  39/  40] time: 192.2409, loss: 0.03093722
[13/50] - ptime: 14.3846 loss: 0.01295502 acc: 0.61333 lr: 0.00090000
Epoch: [14] [   0/  40] time: 193.7422, loss: 0.00114340
Epoch: [14] [   1/  40] time: 194.0811, loss: 0.00179758
Epoch: [14] [   2/  40] time: 194.4205, loss: 0.00259806
Epoch: [14] [   3/  40] time: 194.7620, loss: 0.00640562
Epoch: [14] [   4/  40] time: 195.1003, loss: 0.00023831
Epoch: [14] [   5/  40] time: 195.4432, loss: 0.00106392
Epoch: [14] [   6/  40] time: 195.7826, loss: 0.00279342
Epoch: [14] [   7/  40] time: 196.1219, loss: 0.00116438
Epoch: [14] [   8/  40] time: 196.4607, loss: 0.00064780
Epoch: [14] [   9/  40] time: 196.8018, loss: 0.00806003
Epoch: [14] [  10/  40] time: 197.1407, loss: 0.00247940
Epoch: [14] [  11/  40] time: 197.4811, loss: 0.00111722
Epoch: [14] [  12/  40] time: 197.8215, loss: 0.00096848
Epoch: [14] [  13/  40] time: 198.1612, loss: 0.00110019
Epoch: [14] [  14/  40] time: 198.5013, loss: 0.00546342
Epoch: [14] [  15/

...

Epoch: [34] [  39/  40] time: 502.3701, loss: 0.00000340
[34/50] - ptime: 14.3932 loss: 0.00067235 acc: 0.62000 lr: 0.00072900
Epoch: [35] [   0/  40] time: 503.8906, loss: 0.00001005
Epoch: [35] [   1/  40] time: 504.2270, loss: 0.00016383
Epoch: [35] [   2/  40] time: 504.5675, loss: 0.00166397
Epoch: [35] [   3/  40] time: 504.9064, loss: 0.00003972
Epoch: [35] [   4/  40] time: 505.2450, loss: 0.00012177
Epoch: [35] [   5/  40] time: 505.5853, loss: 0.00004348
Epoch: [35] [   6/  40] time: 505.9235, loss: 0.00005657
Epoch: [35] [   7/  40] time: 506.2623, loss: 0.00000821
Epoch: [35] [   8/  40] time: 506.6025, loss: 0.00010557
Epoch: [35] [   9/  40] time: 506.9404, loss: 0.00022102
Epoch: [35] [  10/  40] time: 507.2787, loss: 0.00110070
Epoch: [35] [  11/  40] time: 507.6181, loss: 0.00038438
Epoch: [35] [  12/  40] time: 507.9583, loss: 0.00031840
Epoch: [35] [  13/  40] time: 508.2972, loss: 0.00000747
Epoch: [35] [  14/  40] time: 508.6370, loss: 0.00015362
Epoch: [35] [  15/

Epoch: [38] [  18/  40] time: 554.2579, loss: 0.00015040
Epoch: [38] [  19/  40] time: 554.5977, loss: 0.00007600
Epoch: [38] [  20/  40] time: 554.9369, loss: 0.00002813
Epoch: [38] [  21/  40] time: 555.2760, loss: 0.00000978
Epoch: [38] [  22/  40] time: 555.6155, loss: 0.00001650
Epoch: [38] [  23/  40] time: 555.9540, loss: 0.00003977
Epoch: [38] [  24/  40] time: 556.2926, loss: 0.00001452
Epoch: [38] [  25/  40] time: 556.6341, loss: 0.00009570
Epoch: [38] [  26/  40] time: 556.9744, loss: 0.00002304
Epoch: [38] [  27/  40] time: 557.3121, loss: 0.00006872
Epoch: [38] [  28/  40] time: 557.6530, loss: 0.00006852
Epoch: [38] [  29/  40] time: 557.9914, loss: 0.00014253
Epoch: [38] [  30/  40] time: 558.3311, loss: 0.00004215
Epoch: [38] [  31/  40] time: 558.6708, loss: 0.00002784
Epoch: [38] [  32/  40] time: 559.0113, loss: 0.00002481
Epoch: [38] [  33/  40] time: 559.3541, loss: 0.00000483
Epoch: [38] [  34/  40] time: 559.6947, loss: 0.00000698
Epoch: [38] [  35/  40] time: 5

Epoch: [41] [  39/  40] time: 605.6692, loss: 0.00002480
[41/50] - ptime: 14.3920 loss: 0.00004335 acc: 0.66000 lr: 0.00065610
Epoch: [42] [   0/  40] time: 607.1550, loss: 0.00004580
Epoch: [42] [   1/  40] time: 607.4930, loss: 0.00010851
Epoch: [42] [   2/  40] time: 607.8347, loss: 0.00000580
Epoch: [42] [   3/  40] time: 608.1743, loss: 0.00004761
Epoch: [42] [   4/  40] time: 608.5132, loss: 0.00002983
Epoch: [42] [   5/  40] time: 608.8525, loss: 0.00000159
Epoch: [42] [   6/  40] time: 609.1932, loss: 0.00005757
Epoch: [42] [   7/  40] time: 609.5338, loss: 0.00003444
Epoch: [42] [   8/  40] time: 609.8734, loss: 0.00001518
Epoch: [42] [   9/  40] time: 610.2128, loss: 0.00000168
Epoch: [42] [  10/  40] time: 610.5511, loss: 0.00011581
Epoch: [42] [  11/  40] time: 610.8908, loss: 0.00022693
Epoch: [42] [  12/  40] time: 611.2301, loss: 0.00000431
Epoch: [42] [  13/  40] time: 611.5698, loss: 0.00018325
Epoch: [42] [  14/  40] time: 611.9094, loss: 0.00006009
Epoch: [42] [  15/

Epoch: [45] [  18/  40] time: 657.5141, loss: 0.00001974
Epoch: [45] [  19/  40] time: 657.8558, loss: 0.00000467
Epoch: [45] [  20/  40] time: 658.2003, loss: 0.00001028
Epoch: [45] [  21/  40] time: 658.5409, loss: 0.00002969
Epoch: [45] [  22/  40] time: 658.8834, loss: 0.00003044
Epoch: [45] [  23/  40] time: 659.2232, loss: 0.00002095
Epoch: [45] [  24/  40] time: 659.5630, loss: 0.00004159
Epoch: [45] [  25/  40] time: 659.9047, loss: 0.00008607
Epoch: [45] [  26/  40] time: 660.2446, loss: 0.00001164
Epoch: [45] [  27/  40] time: 660.5817, loss: 0.00009599
Epoch: [45] [  28/  40] time: 660.9213, loss: 0.00040570
Epoch: [45] [  29/  40] time: 661.2610, loss: 0.00009104
Epoch: [45] [  30/  40] time: 661.6008, loss: 0.00011147
Epoch: [45] [  31/  40] time: 661.9376, loss: 0.00000393
Epoch: [45] [  32/  40] time: 662.2746, loss: 0.00000210
Epoch: [45] [  33/  40] time: 662.6136, loss: 0.00001624
Epoch: [45] [  34/  40] time: 662.9516, loss: 0.00005091
Epoch: [45] [  35/  40] time: 6

Epoch: [48] [  39/  40] time: 708.8525, loss: 0.03492805
[48/50] - ptime: 14.3766 loss: 0.02847550 acc: 0.65000 lr: 0.00065610
Epoch: [49] [   0/  40] time: 710.3318, loss: 0.00896761
Epoch: [49] [   1/  40] time: 710.6709, loss: 0.06000476
Epoch: [49] [   2/  40] time: 711.0111, loss: 0.10077142
Epoch: [49] [   3/  40] time: 711.3504, loss: 0.09941352
Epoch: [49] [   4/  40] time: 711.6905, loss: 0.40142080
Epoch: [49] [   5/  40] time: 712.0291, loss: 1.10966182
Epoch: [49] [   6/  40] time: 712.3673, loss: 0.46392116
Epoch: [49] [   7/  40] time: 712.7071, loss: 4.09213495
Epoch: [49] [   8/  40] time: 713.0471, loss: 0.35735604
Epoch: [49] [   9/  40] time: 713.3851, loss: 0.44108894
Epoch: [49] [  10/  40] time: 713.7243, loss: 0.15523666
Epoch: [49] [  11/  40] time: 714.0628, loss: 0.29911345
Epoch: [49] [  12/  40] time: 714.4035, loss: 0.17490137
Epoch: [49] [  13/  40] time: 714.7431, loss: 0.08992480
Epoch: [49] [  14/  40] time: 715.0830, loss: 0.14548640
Epoch: [49] [  15/

In [4]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放